In [30]:
!pip install spacy==3.0.6

In [31]:
!pip install -U spacy-nightly --pre

In [32]:
!pip install -U pip setuptools wheel

In [33]:
!python -m spacy download en_core_web_trf

In [34]:
!pip install -U spacy transformers

In [35]:
!pip install spacy-transformers

In [36]:
!pip install PyPDF2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/Shareddrives/topnetwork/spacytransformers

/content/drive/Shareddrives/topnetwork/spacytransformers


In [ ]:
import random

from pathlib import Path
import spacy
from spacy.tokens import DocBin, Doc
#from spacy.training.example import Example
from rel_pipe import make_relation_extractor, score_relations
from rel_model import create_relation_model, create_classification_layer, create_instances, create_tensors

In [ ]:
nlp2 = spacy.load("/content/drive/MyDrive/TA_project/progetto_TA/spacytransformers/model-best")

/usr/local/lib/python3.7/dist-packages/spacy/util.py:833: UserWarning: [W095] Model 'it_pipeline' (0.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.2.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
import spacy.cli

spacy.cli.download('it_core_news_lg')

✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_lg')


In [ ]:
nlp_it = spacy.load('it_core_news_lg')

## Test model from http://www.italgiure.giustizia.it/sncass/ judgements

In [ ]:
import PyPDF2

In [ ]:
import re

def clean_jud(text):
    text = text.lower()
    text = text.replace("'"," ")
    #text = text[text.find(' 1.')+3: text.find(' 1.') + 800]
    text = re.sub('art. \d', '',text)
    text = re.sub('artt. \d', '',text)
    text = re.sub("d.p.r. \d", '',text)
    text = re.sub("n. \d", '',text)
    text = re.sub("n. dep \d", '',text)
    text = re.sub('ex \d','',text)
    text = re.sub('comma \d', '',text)

    
    text = text.replace('n.','')
    text = text.replace('cost.','')
    text = text.replace('sent.', '')
    text = text.replace('num.', '')
    text = text.replace('sez.', '')
    text = text.replace('p. e p.', '')
    text = text.replace("d.lgs.", '')
    text = text.replace("cod. pen.", '')
    text = text.replace("cod.proc.pe", '')
    text = text.replace('m.a.e', '')
    text = text.replace('lett.', '')
    
    

    
    text = re.sub(' +', ' ',text)
    return text

In [ ]:
def pdf2str(pdf_link):
    !wget -q '{pdf_link}'  -O 'sentenza.pdf'
    pdfFileObj = open('sentenza.pdf', 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    text = ''
    for p in range(pdfReader.numPages):
        text +=  pdfReader.getPage(p).extractText()
    !rm 'sentenza.pdf'
    return text

In [ ]:
def get_imputati(pdf_link, top_k=3):
    text = pdf2str(pdf_link)
    text = clean_jud(text)

    sentenza = nlp2(nlp_it(text))
    imputati = []
    for value, rel_dict in sentenza._.rel.items():
        for s in sentenza.sents:
            for e in s.ents:
                for b in s.ents:
                    if e.start == value[0] and b.start == value[1]:
                        if rel_dict['DIFENDE'] > rel_dict['GIUDICA']:
                            imputati.append((b.text, rel_dict['DIFENDE']))   
                        
                        
                        else:
                            imputati.append((b.text,  rel_dict['GIUDICA']))
    imputati.sort(key= lambda x: x[1],reverse=True)
                            
    return imputati[:top_k]



In [ ]:
# from cassazione website get a pdf link
pdf_link = "http://www.italgiure.giustizia.it/xway/application/nif/clean/hc.dll?verbo=attach&db=snpen&id=./20211220/snpen@s70@a2021@n46547@tS.clean.pdf"

In [ ]:
get_imputati(pdf_link,3)

[('giovanni carparelli', 0.99405307),
 ('giovanni carparelli', 0.9439249),
 ('giovanni carparelli', 0.71029705)]